In [ ]:
import torch
import torch.nn as nn

from torchvision.models.vgg import vgg16

device = "cuda" if torch.cuda.is_available() else "cpu"

model = vgg16(pretrained=True)

fc = nn.Sequential(
       nn.Linear(512 * 7 * 7, 4096),
       nn.ReLU(),
       nn.Dropout(), # ❷ 드롭아웃층 정의
       nn.Linear(4096, 4096),
       nn.ReLU(),
       nn.Dropout(),
       nn.Linear(4096, 10),
   )

model.classifier = fc
model.to(device)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:07<00:00, 74.7MB/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
# Feature Extractor 설정 (합성곱층 동결)
for param in model.features.parameters():
    param.requires_grad = False

# FC Layer만 학습하도록 설정
for param in model.classifier.parameters():
    param.requires_grad = True

In [ ]:
import tqdm

from torchvision.datasets.cifar import CIFAR10
from torchvision.transforms import Compose, ToTensor, Resize
from torchvision.transforms import RandomHorizontalFlip, RandomCrop, Normalize
from torch.utils.data.dataloader import DataLoader
import torch.optim as optim

transforms = Compose([
   Resize(224),
   RandomCrop((224, 224), padding=4),
   RandomHorizontalFlip(p=0.5),
   ToTensor(),
   Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.243, 0.261))
])

training_data = CIFAR10(root="./", train=True, download=True, transform=transforms)
test_data = CIFAR10(root="./", train=False, download=True, transform=transforms)

train_loader = DataLoader(training_data, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

# ✅ Mixed Precision Training 적용
scaler = torch.cuda.amp.GradScaler()

# ✅ 학습 루프 (속도 최적화)
for epoch in range(10):  # 1 epoch → 5 epochs로 변경 (성능 체크)
    model.train()
    iterator = tqdm.tqdm(train_loader, desc=f"Epoch {epoch+1}/5")

    for data, label in iterator:
        data, label = data.to(device), label.to(device)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():  # Mixed Precision 적용
            preds = model(data)
            loss = criterion(preds, label)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        iterator.set_postfix(loss=loss.item())

"""
for epoch in range(1):
   model.train()
   iterator = tqdm.tqdm(train_loader, desc=f"Epoch {epoch+1}/{30}") # ➊ 학습 로그 출력
   for data, label in iterator:
      data, label = data.to(device), label.to(device)
      optimizer.zero_grad()

      preds = model(data) # 모델의 예측값 출력

      loss = criterion(preds, label)
      loss.backward()
      optimizer.step()

      # ❷ tqdm이 출력할 문자열
      iterator.set_postfix(loss=loss.item())
  """

torch.save(model.state_dict(), "CIFAR_pretrained.pth") # 모델 저장
print("🎯 Model saved as 'CIFAR10_trained.pth'")


model.load_state_dict(torch.load("CIFAR_pretrained.pth", map_location=device))
model.eval()

num_corr = 0



Files already downloaded and verified
Files already downloaded and verified


<ipython-input-6-1630d49f6cce>:27: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Epoch 1/5:   0%|          | 0/1563 [00:00<?, ?it/s]<ipython-input-6-1630d49f6cce>:39: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():  # Mixed Precision 적용
Epoch 10/5: 100%|██████████| 1563/1563 [07:12<00:00,  3.62it/s, loss=0.053]


🎯 Model saved as 'CIFAR10_trained.pth'


<ipython-input-6-1630d49f6cce>:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("CIFAR_pretrained.pth", map_location=device))


✅ Final Accuracy: 0.9131


In [ ]:
import time

num_corr = 0  # 정답 개수 초기화
start_time = time.time()  # 시작 시간 기록

with torch.no_grad():  # 그래디언트 계산 비활성화 (추론 모드)
    for data, label in test_loader:
        data, label = data.to(device), label.to(device)  # 데이터를 GPU로 이동

        output = model(data)  # 모델 예측값
        preds = torch.argmax(output, dim=1)  # 가장 높은 확률을 가진 클래스 선택
        num_corr += preds.eq(label).sum().item()  # 정답 개수 업데이트

end_time = time.time()  # 종료 시간 기록
elapsed_time = end_time - start_time  # 실행 시간 계산

accuracy = num_corr / len(test_loader.dataset)  # 정확도 계산
print(f"✅ Final Accuracy: {accuracy:.4f}")  # 소수점 4자리까지 출력
print(f"⏱️ Inference Time: {elapsed_time:.4f} sec")  # 실행 시간 출력
